In [ ]:
import mnist
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from sklearn.metrics import precision_score, recall_score
from sklearn.model_selection import ParameterGrid
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

print("Empieza entrenamiento con los mejores parametros")
# Cargar los datos de MNIST
X_train, y_train = mnist.train_images(), mnist.train_labels()
X_test, y_test = mnist.test_images(), mnist.test_labels()

# Normalizar las imágenes y aplanarlas para que cada imagen sea un vector de características
X_train = X_train.reshape(X_train.shape[0], -1) / 255.
X_test = X_test.reshape(X_test.shape[0], -1) / 255.

# Definir los parámetros a buscar
param_grid = {'C': [0.1, 1, 10, 100], 'kernel': ['linear', 'rbf']}

# Crear el objeto ParameterGrid
grid = ParameterGrid(param_grid)

# Crear el objeto KFold
kf = KFold(n_splits=5)

# Inicializar las variables para almacenar los mejores resultados
best_params = None
best_precision = 0
best_recall = 0
mean_precisions = []
mean_recalls = []
# Iterar sobre todas las combinaciones de parámetros
for params in grid:
    # Inicializar las variables para almacenar los resultados de cada iteración de validación cruzada
    precision_scores = []
    recall_scores = []
    
    # Iterar sobre cada iteración de validación cruzada
    for train_index, test_index in kf.split(X_train):
        # Dividir los datos en conjuntos de entrenamiento y prueba
        X_train_cv, X_test_cv = X_train[train_index], X_train[test_index]
        y_train_cv, y_test_cv = y_train[train_index], y_train[test_index]
        
        # Entrenar el modelo con los datos de entrenamiento y los parámetros actuales
        svm_model = SVC(**params)
        svm_model.fit(X_train_cv, y_train_cv)
        
        # Evaluar el modelo en el conjunto de prueba
        y_test_pred = svm_model.predict(X_test_cv)
        
        # Calcular la precisión y la exhaustividad y agregarlas a las listas correspondientes
        precision_scores.append(precision_score(y_test_cv, y_test_pred, average='macro'))
        recall_scores.append(recall_score(y_test_cv, y_test_pred, average='macro'))
    
    # Calcular la precisión y la exhaustividad promedio para esta combinación de parámetros
    mean_precision = np.mean(precision_scores)
    mean_recall = np.mean(recall_scores)
    
    # Add the mean precision and mean recall values to their respective lists
    mean_precisions.append(mean_precision)
    mean_recalls.append(mean_recall)
    
    # Print the mean precision and mean recall values for this configuration
    print(f"Parameters: {params}, Precision: {mean_precision}, Recall: {mean_recall}")
    
    # Si la precisión y la exhaustividad son mejores que las mejores hasta ahora, actualizar las variables correspondientes
    if mean_precision > best_precision and mean_recall > best_recall:
        best_params = params
        best_precision = mean_precision
        best_recall = mean_recall
        
# Convert the lists of mean precision and mean recall values to arrays
mean_precisions = np.array(mean_precisions)
mean_recalls = np.array(mean_recalls)

# Imprimir los mejores parámetros encontrados
print("Empieza entrenamiento con los mejores parametros")

# Entrenar el modelo con los mejores parámetros encontrados
svm_model = SVC(**best_params)
svm_model.fit(X_train, y_train)

# Imprimir los mejores parámetros encontrados
print("Best parameters: ", best_params)
# Find the non-dominated points (i.e., the points on the Pareto frontier)
evalpareto = is_pareto_optimal(np.column_stack((mean_precisions, mean_recalls)))

# Plot the precision/recall space
plt.scatter(mean_precisions[~evalpareto], mean_recalls[~evalpareto], color='blue', label='Dominated points')
plt.scatter(mean_precisions[evalpareto], mean_recalls[evalpareto], color='red', label='Pareto frontier')
plt.xlabel('Precision')
plt.ylabel('Recall')
plt.legend()
plt.show()

Empieza entrenamiento con los mejores parametros
Parameters: {'C': 0.1, 'kernel': 'linear'}, Precision: 0.9402590049201074, Recall: 0.9399516800699599
Parameters: {'C': 0.1, 'kernel': 'rbf'}, Precision: 0.9541099165351026, Recall: 0.9542377585404029
Parameters: {'C': 1, 'kernel': 'linear'}, Precision: 0.9326965442059153, Recall: 0.932223900420094
